# Imports

## Packages

In [ ]:
import torch.optim
from torch.utils import data
import argparse
import json
from tqdm import tqdm
import sys
import clip
sys.path.append('/home/guest/Documents/Siraj TM/RSCaMa')
from model.model_encoder_attMamba import Encoder, AttentiveEncoder
from model.model_decoder import DecoderTransformer
from utils_tool.utils import *
from data.LEVIR_CC.LEVIRCC import LEVIRCCDataset
from torch import nn, einsum

from PIL import Image
from imageio import imread
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import time

/home/guest/anaconda3/envs/RSCaMa_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

## Functions

In [2]:
def load_json(path):
    with open(path) as f:
        file = json.load(f)
    f.close()
    return file
def save_json(file,path):
    with open(path,'w') as f:
        json.dump(file,f)
    f.close()
    print("Saved Successfully")
def rem_print(word):
    t_word = word
    for _ in range(100 - len(t_word)):
        word = word + ' '
    print(word,end='\r')

## Data

In [12]:
word_vocab = load_json('assets/vocab.json')

In [ ]:
train_dir = 'data/Levir-CC-dataset/images/train'

# Training

## Imports

In [10]:
from torch.utils import data

## Data

In [38]:
train_loader = data.DataLoader(
                LEVIRCCDataset('CLIP-ViT-B/32','data/Levir-CC-dataset/images', '/home/guest/Documents/Siraj TM/RSCaMa/data/LEVIR_CC/', 'train', '/home/guest/Documents/Siraj TM/RSCaMa/data/LEVIR_CC/tokens/', word_vocab, 42, 1),
                batch_size=64, shuffle=True, num_workers=24, pin_memory=True)

In [41]:
val_loader = data.DataLoader(
                LEVIRCCDataset('CLIP-ViT-B/32','data/Levir-CC-dataset/images', '/home/guest/Documents/Siraj TM/RSCaMa/data/LEVIR_CC/', 'val', '/home/guest/Documents/Siraj TM/RSCaMa/data/LEVIR_CC/tokens/', word_vocab , 42, 1),
                batch_size=64, shuffle=False, num_workers=24, pin_memory=True)

## Model

In [ ]:
model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
encoder = Encoder('CLIP-ViT-B/32')
encoder.fine_tune(True)
encoder_trans = AttentiveEncoder(n_layers=3,
                                        feature_size=[7, 7, 768],
                                        heads=8, dropout=0.1)
decoder = DecoderTransformer(decoder_type='transformer_decoder',embed_dim=768,
                                    vocab_size=len(word_vocab), max_lengths=42,
                                    word_vocab=word_vocab, n_head=8,
                                    n_layers=1, dropout=0.1)

In [ ]:
encoder_optimizer = torch.optim.Adam(params=encoder.parameters(),
                                            lr=1e-4) if True else None
encoder_trans_optimizer = torch.optim.Adam(
    params=filter(lambda p: p.requires_grad, encoder_trans.parameters()),
    lr=1e-4)
decoder_optimizer = torch.optim.Adam(
    params=filter(lambda p: p.requires_grad, decoder.parameters()),
    lr=1e-4)

# Move to GPU, if available
encoder = encoder.cuda()
encoder_trans = encoder_trans.cuda()
decoder = decoder.cuda()
encoder_lr_scheduler = torch.optim.lr_scheduler.StepLR(encoder_optimizer, step_size=5,
                                                            gamma=1.0) if True else None
encoder_trans_lr_scheduler = torch.optim.lr_scheduler.StepLR(encoder_trans_optimizer, step_size=5,
                                                                    gamma=1.0)
decoder_lr_scheduler = torch.optim.lr_scheduler.StepLR(decoder_optimizer, step_size=5,
                                                            gamma=1.0)

## Training Step

In [ ]:
encoder.train()
encoder_trans.train()
decoder.train()


for id,batch_data in enumerate(train_loader):
    
    start_time = time.time()
    
    #Getting Data and moving to GPU if possible
    imgA = batch_data['imgA']
    imgB = batch_data['imgB']
    token = batch_data['token']
    token_len = batch_data['token_len']
    imgA = imgA.cuda()
    imgB = imgB.cuda()
    token = token.cuda()
    token_len = token_len.cuda()
    
    #Feat1 and Feat2
    feat1, feat2 = encoder(imgA, imgB)
    feat = encoder_trans(feat1, feat2)
    scores, caps_sorted, decode_lengths, sort_ind = decoder(feat, token, token_len)
    # Since we decoded starting with <start>, the targets are all words after <start>, up to <end>
    targets = caps_sorted[:, 1:]

AttributeError: 'dict' object has no attribute 'shape'